In [1]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [2]:
import json
import glob

results = glob.glob('custom_val_output/*/predictions.json')
print(results)

['custom_val_output\\mobilenet_v3_large_fpn\\predictions.json', 'custom_val_output\\resnet50_v2_fpn\\predictions.json', 'custom_val_output\\yolov10s_small\\predictions.json', 'custom_val_output\\yolov11m_medium\\predictions.json', 'custom_val_output\\yolov11n_nano\\predictions.json', 'custom_val_output\\yolov11s_small\\predictions.json']


In [3]:
map = json.load(open('coco91to80.json'))
map = {int(v): int(k) for k, v in map.items()} # k: v -> v: k

In [4]:
coco = COCO('datasets/anno/instances_val2017.json')
for result in results:
    with open(result, 'r') as f:
        yolo_preds = json.load(f)
    # Convert YOLO predictions to COCO format
    coco_preds = []
    for pred in yolo_preds:
        cat = pred["category_id"]
        if map.get(cat+1) is not None:
            cat = map[cat+1]
        else:
            assert(False)
            cat = 0
            # try continue
        coco_pred = {
            "image_id": pred["image_id"],
            "category_id":cat,
            "bbox": pred["bbox"],  # [x, y, width, height]
            "score": pred["score"]
        }
        coco_preds.append(coco_pred)
    with open('coco_formatted_predictions.json', 'w') as f:
        json.dump(coco_preds, f)
    print(('#'*50 + '\n')*3)
    dir = result.split("\\")[-2]
    print(f'Directory: {dir}')
    coco_dt = coco.loadRes('coco_formatted_predictions.json')
    cocoEval = COCOeval(coco, coco_dt, 'bbox')
    cocoEval.evaluate()
    cocoEval.accumulate()
    cocoEval.summarize()

loading annotations into memory...
Done (t=0.67s)
creating index...
index created!
##################################################
##################################################
##################################################

Directory: mobilenet_v3_large_fpn
Loading and preparing results...
DONE (t=0.94s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=22.24s).
Accumulating evaluation results...
DONE (t=3.15s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.320
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.514
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.334
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.115
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.354
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.503
 Average Recall     (AR) @[ IoU=0.50